In [4]:
# import libraries

%matplotlib inline 
import os
import random
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')
from bs4 import BeautifulSoup
import re
import string
import unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.util import ngrams
stopwords = stopwords.words('english')
punctuations = string.punctuation
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [7]:
hp = pd.read_csv('sample_data_labelled_cleaned.csv',sep = '|')

In [8]:
def article_preprocessing(st):
    
    st = re.sub(r"&quot;", r" ", st)
    st = re.sub(r"&apos;", r" ", st)
    st = re.sub(r"([.!?])", r" ", st)
    st = re.sub(r"[^a-zA-Z0-9.!?]+", r" ", st)
    st = ' '.join(word for word in st.split(' ') if not word.startswith('httpwww'))
    st = ' '.join(word for word in st.split(' ') if not word.startswith('OrderedD'))
    st = st.replace('type external text','').replace('  ','').replace('In', ' In').replace('externaltype','')
    st = st.replace('externalSource','').replace('type internal text','').replace('external Source','').replace('external external text Source','').replace('internal text','').replace('type internal','')
    st = st.lower()
    
    return st
    

In [9]:
hp['Article'] = hp['Article'].apply(article_preprocessing)

KeyboardInterrupt: 

In [10]:
hpp = pd.read_csv('sample_data_labelled_cleaned.csv', sep = '|')

In [11]:
# Split train data into actual train and validation sets 0.75/0.25

from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(hpp, test_size=0.25, random_state=42)

train_index = train_data['ID']
train_article = train_data['Article']
train_label = train_data['Hyperpartisan']

val_index = val_data['ID']
val_article = val_data['Article']
val_label = val_data['Hyperpartisan']
 
# now, let's build the test dataset. 


print ("Train dataset size is {}".format(len(train_index)))
print ("Val dataset size is {}".format(len(val_index)))


Train dataset size is 3281
Val dataset size is 1094


In [31]:
import pickle
pickle.dump(train_data, open("train_data_split", "wb"))

In [32]:
test_data, val_data_final = train_test_split(val_data, test_size=0.5, random_state=42)

In [33]:
pickle.dump(test_data, open("test_data_split", "wb"))
pickle.dump(val_data, open("val_data_split", "wb"))

In [13]:
# calcualte mean article length

articles = train_article.values
length_of_article = [len(article) for article in articles]
np.mean(length_of_article)

19650.365437366658

In [19]:
def preproc(train_article, n):
    
    token_dataset = []
    all_tokens = []
    values = train_article.values
    for i in range(len(values)):
        if i % 500 == 0:
            print(i)
        article = values[i]
        article = nlp(article, disable=['tagger', 'parser', 'ner'])
        tokenso = [tok.lemma_.strip() for tok in article if tok.lemma_ != '-PRON-']
        tokenso = [tok for tok in tokenso if tok not in stopwords and tok not in punctuations]
     
        # and get a list of all the n-grams,list of n-word tuples 
        grams = list(ngrams(tokenso, n))
        
        # token_dataset is a list of list of tuples
        token_dataset.append(grams)
        
        all_tokens += grams

    return token_dataset, all_tokens

    

In [20]:
# tokenize the training set
train_article_token_dataset, train_all_tokens = preproc(train_article,1)

0
500
1000
1500
2000
2500
3000


In [36]:
pickle.dump(train_article_token_dataset, open("data/train_data_split_preprocessed", "wb"))
pickle.dump(train_all_tokens, open("data/train_all_tokens", "wb"))

In [42]:
val_article = val_data_final["Article"]
val_article_labels = val_data_final["Hyperpartisan"]
val_article_token_dataset, val_all_tokens = preproc(val_article,1)

0
500


In [43]:
pickle.dump(val_article_token_dataset, open("data/val_data_split_preprocessed", "wb"))

In [45]:
test_article = test_data["Article"]
test_article_labels = test_data["Hyperpartisan"]
test_article_token_dataset, test_all_tokens = preproc(val_article,1)

0
500


In [48]:
pickle.dump(test_article_labels, open("data/test_labels", "wb"))
pickle.dump(val_article_labels, open("data/val_labels", "wb"))
pickle.dump(train_label, open("data/train_labels", "wb"))

In [46]:
pickle.dump(test_article_token_dataset, open("data/test_data_split_preprocessed", "wb"))

In [ ]:
test_article = test_data_final["Article"]
val_article_labels = val_data_final["Hyperpartisan"]

In [25]:
train_all_tokens[0]

('Maison',)

In [ ]:
val_data, = train_test_split(val_article_token_dataset, test_size=0.5, random_state=42)

In [39]:
import collections

max_vocab_size = 20000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = collections.Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(train_all_tokens)

In [ ]:
pickle.dump(token2id, open("token2id", "wb"))

In [40]:
print(len(token2id))
print(len(id2token))

20002
20002


In [154]:
# Lets check the dictionary by loading random token from it

random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

In [49]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

# get list of list of idxes
train_data_indices = token2index_dataset(train_article_token_dataset)
val_data_indices = token2index_dataset(val_article_token_dataset)
test_data_indices = token2index_dataset(test_article_token_dataset)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))
print ("Test dataset size is {}".format(len(val_data_indices)))

Train dataset size is 3281
Val dataset size is 547
Test dataset size is 547


In [51]:
pickle.dump(train_data_indices, open("train_data_indexed", "wb"))
pickle.dump(val_data_indices, open("val_data_indexed", "wb"))
pickle.dump(test_data_indices, open("test_data_indexed", "wb"))

In [20]:
MAX_ARTICLE_LENGTH = 19650

import numpy as np
import torch
from torch.utils.data import Dataset

class GroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, article_list, label_list):
        """
        @param data_list: list of review tokens indexes
        @param target_list: list of review targets 

        """
        self.article_list = article_list
        self.label_list = label_list
        assert (len(self.article_list) == len(self.label_list))

    def __len__(self):
        return len(self.article_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx = self.article_list[key][:MAX_ARTICLE_LENGTH]
        label = self.label_list[key]
        return [token_idx, len(token_idx), label]

def group_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    article_list = []
    label_list = []
    length_list = []
    

    
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        article_list.append(padded_vec)
    return [torch.from_numpy(np.array(article_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]


# batchsize better be power of 2
BATCH_SIZE = 128

train_dataset = GroupDataset(train_data_indices, train_label.values.tolist())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=group_collate_func,
                                           shuffle=True)

val_dataset = GroupDataset(val_data_indices, val_label.values.tolist())
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=group_collate_func,
                                           shuffle=True)

